In [ ]:
#default_exp io.psm_reader.psm_reader

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import typing
import pandas as pd

In [ ]:
#export

def translate_other_modification(
    mod_str: str, 
    mod_dict: dict
)->str:
    '''
    Translate modifications of `mod_str` to the AlphaBase 
    format mapped by mod_dict.
    Args:
        mod_str (str): mod list in str format, seperated by ';', 
            e.g. ModA;ModB
        mod_dict (dict): translate mod dict from others to AlphaBase, 
            e.g. for pFind, key=['Phospho[S]','Oxidation[M]'], 
            value=['Phospho@S','Oxidation@M']
    Returns:
        str: new mods in AlphaBase format seperated by ';'. if any
             modification is not in `mod_dict`, return pd.NA.
    '''
    if not mod_str: return ""
    ret_mods = []
    for mod in mod_str.split(';'):
        if mod in mod_dict:
            ret_mods.append(mod_dict[mod])
        else:
            return pd.NA
    return ";".join(ret_mods)

def keep_modifications(
    mod_str: str, 
    mod_set: set
)->str:
    '''
    Check if modifications of `mod_str` are in `mod_set`.
    Args:
        mod_str (str): mod list in str format, seperated by ';', 
            e.g. Oxidation@M;Phospho@S.
        mod_set (set): mod set to check
    Returns:
        str: original `mod_str` if all modifications are in mod_set 
             else pd.NA.
    '''
    if not mod_str: return ""
    for mod in mod_str.split(';'):
        if not mod in mod_set:
            return pd.NA
    return mod_str



# PSMReaderBase 

`PSMReaderBase` is the base abstract class for all readers. It defines the basic procedures for loading other search engine results into AlphaBase format.

The main entry method is `load(filename)`, and it will generate `self._psm_df` (or property `self.psm_df`) after `load()`.

In `load()` method, we designed five steps to load result files in to AlphaBase format:

1. `origin_df = self._load_file(filename)`. We load result files into a dataframe without doing any file conversion. As different search engines have different file format, **all subclass of `PSMReaderBase` need to re-implement this method**.

2. `self._translate_columns(origin_df)`. We translate columns in `origin_df` into AlphaBase columns by `self.column_mapping`. `self.column_mapping` provides a flexible way for developers to extract their required columns.

3. `self._load_modifications(origin_df)`. As different search engines have different representation of modifications. We use this method to extract the modifications into `self._psm_df['mods']` and `self._psm_df['mod_sites']`. Note that the modification names are still in other search engines' format.

4. `self._translate_modifications()`. Convert modification names into AlphaBase names (`unimod_name@AA`). For most of the search engines, we need a dict (`self.modification_mapping`) to map search engine modification format into AlphaBase (`unimod_name@AA`, `unimod_name` is <umod:mod title=...> in the unimod xml file). **All subclass of `PSMReaderBase` need to re-implement this method**.

5. `self._post_process(filename, origin_df)`. Any required post-processing steps. For example, we remove unknown modifications here.

In [ ]:
#export

class PSMReaderBase(object):
    def __init__(self, 
        modification_mapping:dict = None
    ):
        # modification_mapping=dict[str, str]: 
        #     key:   mod names of other search engines
        #     value: mod names in AlphaBase
        # It is used to convert mods of other engines 
        # to AlphaBase format. Different search engines
        # have different mod names.

        self.modification_mapping = modification_mapping

        self.column_mapping = {
            'sequence': 'NakedSequence',
            # AlphaBase does not need 'modified_sequence',
            # but it will get 'mods', 'mod_sites' from it
            'modified_sequence': 'ModifiedSequence',
            'charge': 'Charge',
            # If the value is a list, check if one of the columns exist
            # and get 'proteins' from that column
            'proteins':['Proteins','UniprotIDs'],
            # Similar to 'proteins'
            'uniprot_ids':['UniprotIDs','UniprotIds'],
            # Similar to 'proteins'
            'genes': ['Genes','Gene Names','Gene names'],
            'fdr': 'fdr',
        } # Add more columns for subclasses of different tasks

        self._psm_df = pd.DataFrame()
        self.keep_all_psm = False # keep all PSMs for percolator or ...
        
    @property
    def psm_df(self):
        return self._psm_df

    def load(self, filename):
        """
        This is the main entry function of PSM readers.
        Args:
            filename (str): [description]
        """
        origin_df = self._load_file(filename)
        self._translate_columns(origin_df)
        self._load_modifications(origin_df)
        self._translate_modifications()
        self._post_process(filename, origin_df)

    def _load_file(self, filename:str)->pd.DataFrame:
        """
        Load original dataframe from PSM filename. 
        Different search engines may store PSMs in different ways:
        tsv, csv, HDF, XML, ...

        Args:
            filename (str): psm filename

        Raises:
            NotImplementedError: Subclasses must re-implement this method

        Returns:
            pd.DataFrame: dataframe loaded
        """
        raise NotImplementedError(
            f'"{self.__class__}" must re-implement "_load_file()"'
        )

    def _translate_columns(self, origin_df:pd.DataFrame):
        """
        Translate the dataframe from other search engines 
        to AlphaBase format

        Args:
            origin_df (pd.DataFrame): df of other search engines

        Returns: 
            None. Add information inplace into self._psm_df
        """
        self._psm_df = pd.DataFrame()
        for col, map_col in self.column_mapping.items():
            if isinstance(map_col, str):
                if map_col in origin_df.columns:
                    self._psm_df[col] = origin_df[map_col]
                else:
                    self._psm_df[col] = pd.NA
            else:
                for other_col in map_col:
                    if other_col in origin_df.columns:
                        self._psm_df[col] = origin_df[other_col]
                        break
                if col not in self._psm_df.columns:
                    self._psm_df[col] = pd.NA
        origin_df['nAA'] = origin_df[self.column_mapping['sequence']].str.len()
        self._psm_df['nAA'] = origin_df['nAA']
    

    def _load_modifications(self, origin_df:pd.DataFrame):
        """
        How to read modification information from 'origin_df'. 
        Some of search engines use modified_sequence, some of them
        use additional columns to store modifications. 

        Args:
            origin_df (pd.DataFrame): dataframe of original search engine.

        Returns: 
            None. Add information inplace into 
            self._psm_df['mods'], self._psm_df['mod_sites'].
        """
        raise NotImplementedError(
            f'"{self.__class__}" must re-implement "_load_modifications()"'
        )

    def _translate_modifications(self):
        '''
        Translate modifications to AlphaBase format.

        Raises: KeyError if `mod` in `mod_names` is 
            not in `self.modification_mapping`
        '''
        self._psm_df.mods = self._psm_df.mods.apply(
            translate_other_modification, 
            mod_dict=self.modification_mapping
        )

    def _post_process(self, 
        filename:str, origin_df:pd.DataFrame
    ):
        """
        Remove unknown modifications and perform other post processings, 
        e.g. loading fragments for AlphaQuant or AlphaDeep

        Args:
            filename (str): psm filename
            origin_df (pd.DataFrame): the loaded original df
        """
        origin_df = origin_df[
            ~self._psm_df['mods'].isna()
        ].reset_index(drop=True)
        
        self._psm_df = self._psm_df[
            ~self._psm_df['mods'].isna()
        ].reset_index(drop=True)

    def filter_psm_by_modifications(self, include_mod_list = [
        'Oxidation@M','Phospho@S','Phospho@T','Phospho@Y','Acetyl@Protein N-term'
    ]):
        '''
            Only keeps peptides with modifications in `include_mod_list`.
        '''
        mod_set = set(include_mod_list)
        self._psm_df.mods = self._psm_df.mods.apply(keep_modifications, mod_set=mod_set)
        
        self._psm_df.dropna(
            subset=['mods'], inplace=True
        )
        self._psm_df.reset_index(drop=True, inplace=True)


# PSMReaderProvider

To make it easier to create different readers, we design a `Provider` or `Factory` called `PSMReaderProvider` to manage all reader classes. `PSMReaderProvider` is instantiated as a global object `psm_reader_provider`. 

After a subclass of `PSMReaderBase` is defined, for example `AlphaPeptReader`, we can then register it in to `psm_reader_provider` by using `psm_reader_provider.register_reader('alphapept', AlphaPeptReader)`. Once we are going to use it, we just need to create a `AlphaPeptReader` object with `psm_reader_provider.get_reader('alphapept')`.

In [ ]:
#export
class PSMReaderProvider:
    def __init__(self):
        self.reader_dict = {}

    def register_reader(self, reader_name, reader_class):
        self.reader_dict[reader_name.lower()] = reader_class

    def get_reader(self, reader_name, modification_mapping:dict=None
    )->PSMReaderBase:
        return self.reader_dict[reader_name.lower()](modification_mapping)

psm_reader_provider = PSMReaderProvider()

As we have loaded all readers in `psm_reader_provider` within alphabase.io.psm_reader.\__init__.py, we can easily access all readers by `psm_reader_provider`.

In [ ]:
# we have loaded all readers in psm_reader_provider
# alphabase.io.psm_reader.__init__.py
from alphabase.io.psm_reader import psm_reader_provider

psm_reader_provider.get_reader('alphapept')

In [ ]:
psm_reader_provider.get_reader('maxquant')

In [ ]:
psm_reader_provider.get_reader('pfind')

In [ ]:
psm_reader_provider.get_reader('diann')

In [ ]:
psm_reader_provider.get_reader('spectronaut')

In [ ]:
psm_reader_provider.get_reader('openswath')